In [1]:
import numpy as np
import scipy.linalg as sla

## Decomposição LU

Nas aulas anteriores, nós trabalhamos o método de escalonamento gaussiano como forma de resolver sistemas de equações lineares não triviais.

Como lembramos, o método tinha dois passos: o primeiro, de eliminação ou escalonamento, tinha como intenção triangularizar a matriz de coeficientes $A$ a partir de uma série de operações que eram aplicados tanto à matriz quanto ao vetor de termos independentes $\vec{b}$, resultando, dessa forma, numa matriz equivalente, triangular superior, e num novo vetor de termos independentes; e o segundo era a solução direta do sistema triangular superior por substituição regressiva.

Embora esse método, principalmente e versão com pivotamento, seja seguro no sentido de garantir a solução correta, ele apresenta duas limitações:

1. a etapa de escalonamento da eliminação gaussiana, principalmente com pivotamento, tem elevado custo computacional, principalmente para grandes sistemas de equações, como ilustra o quadro abaixo, que ilustra o número de _flops_ (_floating-point operations per second_  - operações de ponto flutuante por segundo) para a eliminação gaussiana. Para comparar, basta considerar que a eliminação gaussiana requer $\mathcal{O}(n^3/3)$ operações matemáticas para determinar $\vec{x}$. No entanto, resolver um sistema triangular superior ou inferior, requer somente $\mathcal{O}(2n^2)$. O quadro abaixo dimensiona, a título de ilustração, a redução de custo computacional de se trabalhar com sistemas triangulares se comparados com a eliminação gaussiana.


|$n$ | $n^3/3$ | $2n^2$| redução de custo computacional|
| :-:|:-----:|:-------:|:-:|
|10| $3,3 \cdot 10^2$ | $2 \cdot 10^2$ | 40% |
|100|  $3,3 \cdot 10^5$ | $2 \cdot 10^4$ | 94% |
|1000|  $3,3 \cdot 10^8$ | $2 \cdot 10^6$ | 99,4% |


2. esse custo computacional torna essa abordagem particularmente ineficiente para encontrar a solução de diversos sistemas que tenham mesma matriz de coeficientes, mas diferentes vetores de termos independentes, uma vez que, em todos os sistemas, todo o processo terá de ser inteiramente aplicado.

Uma alternativa à eliminação gaussiana, que trabalharemos essa semana, é a utilização da fatoração ou decomposição LU, que basicamente se propõe a decompor a matriz $\mathbf{A}$ em duas matrizes $\mathbf{L}$, triangular inferior, e $U$, triangular superior, de forma que

$$
\mathbf{A} = \mathbf{LU}
$$

De forma bem simplificada, a solução de sistemas de equações lineares por decomposição LU tem dois passos bem definidos:

1. __decomposição LU:__ $\mathbf{A}$ é fatorada em duas matrizes triangulares superior e inferior;
2. __substituição:__ $\mathbf{L}$ e $\mathbf{U}$ são usadas para determinar o vetor solução $x$, considerando, basicamente, que

\begin{eqnarray*}
\mathbf{A}\vec{x} = \vec{b}~~~ \Rightarrow~~~ \mathbf{LU}\vec{x} = \vec{b}
\end{eqnarray*}

Fazendo o sistema intermediário $\mathbf{U}\vec{x} = \vec{y}$, obtemos outro sistema intermediário, $\mathbf{L}\vec{y} = \vec{b}$. E podemos resolver o sistema original em duas etapas:



* primeiro, se resolve o sistema intermediário $\mathbf{L}\vec{y} = \vec{b}$, para encontrar o vetor $\vec{y}$ usando substituição progressiva;
    
* em seguida, o vetor $\vec{y}$ é usado como vetor de termos independentes do outro sistema intermediário, $\mathbf{U}\vec{x} = \vec{y}$, e, usando substituição retroativa, se encontra o vetor solução desejado.
    
A figura abaixo ilustra o método.

<img src="LU.png" width=400 height=auto align="center">

Essa abordagem diminui sensivelmente o custo computacional se comparado a eliminação gaussiana, uma vez que todas as operações, após a decomposição, serão feitas em um sistema triangular inferior ou superior (e como vimos acima isso reduz drásticamente o custo computacional), além de permitir que, uma vez que a decomposição se realiza somente na matriz de coeficientes, diferentes sistemas com mesma matriz de coeficientes mas diferentes vetores de termos independentes possam ser resolvidos, com uma única execução da decomposição.

## Estratégias de Decomposição LU

Não existe uma única forma de obter as matrizes $\mathbf{L}$ e $\mathbf{U}$. Na verdade, existem inúmeras possíveis abordagens, que se diferenciam, basicamente, por certas restrições usadas para definir algumas características das matrizes $\mathbf{L}$ ou $\mathbf{U}$, e que acabam diferenciando as abordagens. No geral, podemos pensar nas seguintes abordagens:

1. a decomposição derivada da eliminação gaussiana, que pode ser trabalhada de duas formas:
    * a decomposição de Doolittle, que tem como restrição o fato de diagonal principal da matriz triangular inferior ter todos os elementos iguais a 1;
    * a decomposição de Crout, que tem como restrição a diagonal principal da matriz triangular superior com todos os elementos unitários e
2. a decomposição de Choleski, em que a matriz triangular inferior é a transposta da matriz triangular superior.

O quadro abaixo sumariza esses métodos.

| Método | Características |
|:-----:|:-:|
| Doolittle | eliminação gaussiana com diagonal principal da matriz triangular inferior igual a 1 ($L_{ii} = 1$)  |
| Crout | eliminação gaussiana com  diagonal principal da matriz triangular superior igual a 1 ($U_{ii} = 1$) 
| Choleski | matrizes triangulares superior e inferior transpostas ($\mathbf{L} = \mathbf{U^T}$)

### Decomposição LU a Partir da Eliminação Gaussiana

A própria eliminação gaussiana pode ser usada de maneira relativamente simples para implementarmos a decomposição LU. Para entendermos o funcionamento, trabalharemos com a decomposição de Doolittle, que tem como característica a diagonal principal da matriz $\mathbf{L}$ com valores iguais a 1.

Nesse processo, temos dois passos:

* o escalonamento da matriz $\mathbf{A}$ (não a matriz estendida) nos retornará a matriz $\mathbf{U}$;

* os multiplicadores serão os elementos da matriz $\mathbf{L}$ (no caso, os elementos abaixo da diagonal principal).

Ainda no processo de escalonamento, como nós vimos para a eliminação gaussiana, pode ser necessário a realização de permutas de linhas para garantir que o maior valor de cada coluna esteja na diagonal principal, por meio de escalonamento parcial. 

Nesse processo, utilizaremos uma abordagem de uso de uma matriz de permutação $\mathbf{P}$, que é uma matriz formada apenas de zeros e uns, sendo o valor de apenas um elemento por linha e por coluna que igual a um, marcando, justamente que linha foi permutada e cujo objetivo é gerar uma permutação dos elementos de um vetor ou entre linhas ou colunas de uma matriz. 

No caso de usarmos a matriz de permutação, teremos uma pequena variação da expressão que representa o sistema, que é

$$
\mathbf{P A}\vec{x} = \mathbf{P} \vec{b},
$$

sendo no caso, a decomposição,

$$
\mathbf{P A} = \mathbf{LU}
$$

Vamos considerar o exemplo a seguir pra gente entender o funcionamento dessa abordagem. Para nosso entendimento a linha $i$ será referenciada como $E_i$.

$$
\mathbf{A} = \underbrace{\begin{bmatrix}\color{red}{1}&1&0\\\color{red}{2}&1&-1\\\color{red}{3}&-1&-1\end{bmatrix}}_{P = \begin{bmatrix}1&0&0\\0&1&0\\0&0&1\end{bmatrix}}
\xrightarrow{\text{troca linhas 1 e 3}}
\underbrace{\begin{bmatrix}\color{red}{3}&-1&-1\\\color{red}{2}&1&-1\\\color{red}{1}&1&0\end{bmatrix}}_{P = \begin{bmatrix}0&0&1\\0&1&0\\1&0&0\end{bmatrix}}\\[4cm]
\xrightarrow{\begin{align*}E^{(1)}_2 &= E_2-(2/3)E_1\\E^{(1)}_3 &= E_3-(1/3)E_1\end{align*}} 
\underbrace{\begin{bmatrix}\color{red}{3}&-1&-1\\\color{red}{0}&\frac{5}{3}&\frac{1}{3}\\\color{red}{0}&\frac{4}{3}&\frac{1}{3}\end{bmatrix}}_{P = \begin{bmatrix}0&0&1\\0&1&0\\1&0&0\end{bmatrix}}\\[4cm]
$$

$$
\mathbf{A^{(1)}} =
\underbrace{\begin{bmatrix}3& \color{red}{-1}&-1\\0& \color{red}{\frac{5}{3}}&\frac{1}{3}\\0& \color{red}{\frac{4}{3}}&\frac{1}{3}\end{bmatrix}}_{P = \begin{bmatrix}0&0&1\\0&1&0\\1&0&0\end{bmatrix}}
\xrightarrow{E^{(2)}_3 = E^{(1)}_3-(4/5)E_2^{(1)}}
\underbrace{\begin{bmatrix}3& \color{red}{-1}&-1\\0& \color{red}{\frac{5}{3}}&\frac{1}{3}\\0& \color{red}{0}&\frac{3}{5}\end{bmatrix}}_{P = \begin{bmatrix}0&0&1\\0&1&0\\1&0&0\end{bmatrix}}
$$

Assim, obtemos a matriz de permutação

$$
P = \begin{bmatrix}0&0&1\\0&1&0\\1&0&0\end{bmatrix}
$$

a matriz $\mathbf{U}$ como a matriz escalonada

$$
\mathbf{U} = \begin{bmatrix}3& -1&-1\\0&\frac{5}{3}&\frac{1}{3}\\0&0&\frac{3}{5}\end{bmatrix}
$$

e a matriz $\mathbf{L}$, usando os multiplicadores de cada operação de cada linha

$$
\mathbf{L} = \begin{bmatrix}1& 0&0\\\frac{2}{3}&1&0\\\frac{1}{3}&\frac{4}{5}&1\end{bmatrix}
$$

#### Implementação do algoritmo de Doolittle

Considerando as matrizes generalizadas,

\begin{equation}
\mathbf{A} = \begin{bmatrix}a_{11}&a_{12}&\ldots&a_{1n}\\a_{21}&a_{22}& \cdots&a_{2n}\\\vdots& \vdots&\ddots& \vdots\\a_{n1}&a_{n2}&\ldots&a_{nn}\end{bmatrix},
\mathbf{L} = \begin{bmatrix}1&0&\ldots&0\\l_{21}&1& \cdots&0\\\vdots& \vdots&\ddots& \vdots\\l_{n1}&l_{n2}&\ldots&1\end{bmatrix}, \\[2cm]
\mathbf{U} = \begin{bmatrix}u_{11}&u_{12}&\ldots&u_{1n}\\0&u_{22}& &u_{2n}\\\vdots& \vdots&\ddots& \vdots\\0&0&\ldots&u_{nn}\end{bmatrix},
\mathbf{P} = \begin{bmatrix}p_{11}&p_{12}&\ldots&p_{1n}\\p_{21}&\ddots& &p_{2n}\\\vdots& &\ddots& \vdots\\p_{n1}&\ldots&\ldots&p_{nn}\end{bmatrix} \\[2cm]
\end{equation}

o algoritmo de Doolittle é

| Algoritmo de Doolittle | |
| --:   | :-- |
|___Entrada.___ | Matriz $\mathbf{A}$|
|___Passo  1.___  | Inicialize $\mathbf{L = P = I}$, de dimensão $n \times n$, e $\mathbf{U = A}$ |
| ___Passo  2.___  | Para $j = 1, \ldots, n-1$ |
|     | $\phantom{--}$ Se $|u_{pj}| = \max{(|u_{(j:n,j)}|)}> u_{jj}, \text{ com } j \leq p < n$ |
|     | $\phantom{----}$ Permutar linha $U_j$ com linha $U_{p}$ |
|   | $\phantom{----}$ Permutar linha $P_j$ com linha $P_{p}$ |
| ___Passo 3.___  | $\phantom{----}$ Para $i = j+1, \ldots, n$ |
|     | $\phantom{------}$ $l_{ij} = \dfrac{u_{ij}}{u_{jj}}$ |
|    | $\phantom{------}$ $U_{i}=U_{i} - l_{ij} U_{j}$ (em que $U_i, U_j$ representam as linhas $i$ e $j$ da matriz $\mathbf{U}$, respectivamente) 

### Solução de Equações Após Decomposição LU

Para solucionar o sistema completo, uma vez que temos as matrizes $\mathbf{P}$, $\mathbf{L}$ e $\mathbf{U}$, é, como comentamos anteriormente, resolver os dois sistemas intermediários:

$$\mathbf{L}\vec{y} = \mathbf{P}\vec{b}~~~~~~\mathbf{U}\vec{x} = \vec{y}$$

Notemos que $\mathbf{P}$ multiplica o vetor de termos independentes para que possamos a versão modificada pelo pivotamento. 

Como os dois sistemas são triangulares, e nós já implementamos, em aulas anteriores, funções para resolver esses sistemas, basta, então, construir um implementação integrando todas as "partes" do processo. Considerando que usaremos essas mesmas funções já implementadas, `sist_lin_tri_sup()` e `sist_lin_tri_inf()`, o algoritmo a seguir descreve isso

| Solução de sistemas com decomposição LU | |
| --:   | :-- |
|___Entrada.___  | Matrizes $\mathbf{P}$, $\mathbf{L}$ e $\mathbf{U}$ e vetor de termos indendentes $\vec{b}$|
| ___Passo  1.___  | Resolução do sistema triangular inferior $\mathbf{L}\vec{y} = \mathbf{P}\vec{b}$, usando a função `sist_lin_tri_inf(L,Pb)`, que retornará o vetor $\vec{y}$
| ___Passo  2.___  | Resolução do sistema triangular superior $\mathbf{U}\vec{x} = \vec{y}$, usando a função `sist_lin_tri_sup(U,y)`, que retornará o vetor $\vec{x}$

### Decomposição LU usando `Scipy`

A biblioteca `scipy` possui três funções, no submódulo `linalg`, sobre decomposição LU:

* [lu](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu.html): calcula a decomposição LU com pivotamento de uma matriz $\mathbf{A}$, decompondo no formato $\mathbf{A = PLU}$. Esse método retorna, se o parâmetro `permute_l == False`, a matriz de permutação $\mathbf{P}$, a matriz $\mathbf{L}$, com elementos unitários na diagonal principal, e a matriz $\mathbf{U}$, e se `permute_l == True`, retorna a matriz $\mathbf{PL}$ e a matriz $\mathbf{U}$

* [lu_factor](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu_factor.html): também calcula a decomposição LU com pivotamento de uma matriz $\mathbf{A}$, decompondo no formato $\mathbf{A = PLU}$, retornando, diferentemente da função anterior, uma matriz $\mathbf{[LU]}$, cujos elementos de $\mathbf{U}$ na parte triangular superior e de $\mathbf{L}$ na triangular inferior (os elementos unitários da diagonal de $\mathbf{L}$ não são armazenados) e um vetor `piv` contendo os índices do pivotamento, indicando que a linha `i` foi permutada com a linha `piv[i]`.

* [lu_solve](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu_solve.html): soluciona um sistema $\mathbf{A}\vec{x} = \vec{b}$, a usando fatoração LU. Como entrada, recebe uma tupla formada pelas matrizes $\mathbf{[LU]}$ e `piv` (saída da função `lu_factor`) e vetor de termos independentes $\vec{b}$.

Em resumo, podemos pensar em dois casos sobre o uso dessas funções:

1. se a necessidade for realizar a decomposição LU, para encontrar as partes da decomposição, a função para se usar é `lu`;

2. se a necessidade for resolver o sistema, então é preciso usar a função `lu_factor`, e usar sua saída como entrada para a função `lu_solve`.

Para podermos visualizar melhor o uso dessas funções, vamos resolver o sistema

$$
  \begin{split}
    x_1 + x_2 +x_3  &= -2\\
    2x_1 + x_2 -x_3  &= 1\\
    2x_1 - x_2 +x_3  &= 3\\
  \end{split}
$$

cuja matriz de coeficientes e vetor de termos independentes pode ser escrito como

In [2]:
A = np.array([[1,1,1],[2,1,-1],[2,-1,1]])
A

array([[ 1,  1,  1],
       [ 2,  1, -1],
       [ 2, -1,  1]])

In [3]:
b = np.array([-2,1,3])
b

array([-2,  1,  3])

Se quisermos saber quem são as matrizes $\mathbf{P}$, $\mathbf{L}$ e $\mathbf{U}$, fazemos

In [4]:
P,L,U = sla.lu(A)

In [5]:
P

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [6]:
L

array([[ 1.  ,  0.  ,  0.  ],
       [ 1.  ,  1.  ,  0.  ],
       [ 0.5 , -0.25,  1.  ]])

In [8]:
U

array([[ 2.,  1., -1.],
       [ 0., -2.,  2.],
       [ 0.,  0.,  2.]])

In [9]:
P@L@U

array([[ 1.,  1.,  1.],
       [ 2.,  1., -1.],
       [ 2., -1.,  1.]])

Já se desejamos encontrar o vetor solução do sistema, precisamos fazer a sequência

In [10]:
LU, piv = sla.lu_factor(A)

In [11]:
LU

array([[ 2.  ,  1.  , -1.  ],
       [ 1.  , -2.  ,  2.  ],
       [ 0.5 , -0.25,  2.  ]])

In [12]:
piv

array([1, 2, 2], dtype=int32)

In [13]:
x = sla.lu_solve((LU,piv),b)
x

array([ 1., -2., -1.])

E, testando, de fato temos

In [14]:
A@x

array([-2.,  1.,  3.])